In [2]:
pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys

import csv
import pandas as pd
import re

### WebDriver 실행 및 리뷰 페이지 열기

In [4]:
# URL 파일 불러오기
df = pd.read_csv("C:/Users/USER/Desktop/airbnb_review_urls.csv")
df.head()

,url
0,https://www.airbnb.co.kr/rooms/121074299044943...
1,https://www.airbnb.co.kr/rooms/123539777972413...
2,https://www.airbnb.co.kr/rooms/132685899995183...
3,https://www.airbnb.co.kr/rooms/124892954225532...
4,https://www.airbnb.co.kr/rooms/112384305348536...


In [7]:
# Emoji package
!pip install emoji
from emoji import core

Defaulting to user installation because normal site-packages is not writeable


In [22]:
def cleaned_text(text):
    # 이모지 및 특수 문자 제거 (한글, 영문, 숫자만 남김)
    text = text.replace("\xa0", " ")  # 특수 공백을 일반 공백으로 변환
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)  # 한글, 영어, 숫자, 공백만 남기기
    return text

In [33]:
# WebDriver 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 총 데이터 개수
total_num = 0

# for문 이용해서 리뷰 페이지 열기
for i in range(750, 1000):
    url = df.loc[i, 'url']
    driver.get(url)

    time.sleep(5) # 페이지 로딩 대기

    # 페이지가 완전히 로드될 때까지 대기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    print("페이지 로드 완료")

    # 페이지 소스 가져와서 파싱
    soup = BeautifulSoup(driver.page_source, "lxml")

    # 리뷰 포함된 div 찾기 (답글 제외)
    review_divs = soup.find_all('div', {'class': 'r1bctolv atm_c8_1sjzizj atm_g3_1dgusqm atm_26_lfmit2_13uojos atm_5j_1y44olf_13uojos atm_l8_1s2714j_13uojos dir dir-ltr'})

    # 엑셀 파일에 review 추가하기
    with open("C:/Users/USER/Desktop/review output.csv", 'a', newline='') as f:
        writer = csv.writer(f)

        # 중복 제거
        num_reviews = len(review_divs)

        if num_reviews >= 12:
            review_divs = review_divs[6:]
            num_reviews -= 6

        else:
            review_divs = review_divs[num_reviews // 2:]
            num_reviews //= 2

        print(num_reviews)
        total_num += num_reviews
            
        # 엑셀 파일에 데이터 추가   
        for div in review_divs:
            spans = div.find_all('span', class_='l1h825yc atm_kd_19r6f69_24z95b atm_kd_19r6f69_1xbvphn_1oszvuo dir dir-ltr')
            for span in spans:
                review_text = span.text.strip()
                # emoji 제거
                clean_text = core.replace_emoji(review_text, replace="")
                # 특수문자 제거
                clean_review = cleaned_text(clean_text)
                if clean_review:  # 빈 문자열 방지
                    print("추출된 리뷰:", clean_review)
                    writer.writerow([clean_review])

    time.sleep(2)
    print("완료", i+1)

print("데이터 총 개수 :", total_num)

페이지 로드 완료
24
추출된 리뷰: 숙소 사진에서 보이는거와 같이 예쁘거 깔끔합니당침대 편안하고 베개 폭신해서 진짜 편합니다화장실도 넓고 수건도 넉넉했어요너무너무 편하게 잘 지내다 가서 좋은 기억으로 남는 숙소입니다 다음에 또 방문할게요
추출된 리뷰: 깨끗하고 따뜻해서 편히 쉴 수 있었습니다 체크인 시간이 늦은 대신 체크아웃 시간 또한 늦어서 괜찮았습니다 재방문의사 있습니다
추출된 리뷰: 생각보다 넓고 건물 자체도 깔끔해서 지내기 좋았어요 침구도 깨끗하고 편했어요 지하철역과 버스정류장도 가깝습니다 경의선숲길 산책로가 바로 옆인데 산책 못해서 아쉽네요 다음에 또 묵고 싶어요
추출된 리뷰: 위치가 너무 좋았어요 홍대도 가깝고 여의도 한강동원에도 너무 가까워서 편했어요숙소 근처에 편의점도 있고 버스 정류장도 가까워요방은 많이 넓은 편이 아니지만 깔끔하고 좋았어요텔레비전으로 넷플릭스 볼 수 있어요
추출된 리뷰: 빌라 입구와 복도 방 모두 청결합니다 비품용 충전기가 있어서 편리했고 화장실도 매우 넓습니다 방은 쾌적하며 이불이 두꺼워 포근하였습니다 또한 주방시설도 매우 잘 되어있으며 분리수거함이 복도에 설치되어 있어서 편하게 처리할 수 있었습니다
추출된 리뷰: 대흥역이랑 가깝고 근처에 버스정류장도 있어서 편하게 이동할 수 있었어요 세 명이 이용했는데 소파베드를 펼쳐주셔서 좁지 않게 쉴 수 있었습니다 모든 시설이 깔끔하고 특히 화장실에 샤워부스가 분리되어 있어서 좋았습니다 호텔 수건같은 수건이 넉넉하게 구비되어 있어서 편하게 씻었습니다
추출된 리뷰: 아파트는 매우 멋지고 아늑하며 겨울에는 욕실이 조금 춥고 준은 열정적으로 협조적이지만 오후 7시에 체크인해야 했기 때문에 꽤 오래 기다려야 했고 섭씨 4도 날씨는 힘들었습니다 하지만 저는 여전히 이곳을 아주 좋은 가격에 경험할 가치가 있는 곳이라고 생각합니다
추출된 리뷰: 즐거운 시간을 보냈습니다 아파트는 도착 시 흠잡을 데 없이 깨끗했고 소지품을 보관할 수 있는 공간이 넉넉했습니다 설명은 우리가 얻은 것과 일치했지만 우리는